In [1]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import sklearn
import sys
import tensorflow as tf
import time

from tensorflow import keras

In [2]:
input_filepath = "./shakespeare.txt"
text = open(input_filepath, "r").read()

print(len(text))
print(text[0:100])

1115394
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [3]:
vocab = sorted(set(text))

print(len(vocab))
print(vocab)

65
['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [4]:
char2idx = {char:idx for idx, char in enumerate(vocab)}

print(char2idx)

{'\n': 0, ' ': 1, '!': 2, '$': 3, '&': 4, "'": 5, ',': 6, '-': 7, '.': 8, '3': 9, ':': 10, ';': 11, '?': 12, 'A': 13, 'B': 14, 'C': 15, 'D': 16, 'E': 17, 'F': 18, 'G': 19, 'H': 20, 'I': 21, 'J': 22, 'K': 23, 'L': 24, 'M': 25, 'N': 26, 'O': 27, 'P': 28, 'Q': 29, 'R': 30, 'S': 31, 'T': 32, 'U': 33, 'V': 34, 'W': 35, 'X': 36, 'Y': 37, 'Z': 38, 'a': 39, 'b': 40, 'c': 41, 'd': 42, 'e': 43, 'f': 44, 'g': 45, 'h': 46, 'i': 47, 'j': 48, 'k': 49, 'l': 50, 'm': 51, 'n': 52, 'o': 53, 'p': 54, 'q': 55, 'r': 56, 's': 57, 't': 58, 'u': 59, 'v': 60, 'w': 61, 'x': 62, 'y': 63, 'z': 64}


In [5]:
idx2char = np.array(vocab)

print(idx2char)

['\n' ' ' '!' '$' '&' "'" ',' '-' '.' '3' ':' ';' '?' 'A' 'B' 'C' 'D' 'E'
 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O' 'P' 'Q' 'R' 'S' 'T' 'U' 'V' 'W'
 'X' 'Y' 'Z' 'a' 'b' 'c' 'd' 'e' 'f' 'g' 'h' 'i' 'j' 'k' 'l' 'm' 'n' 'o'
 'p' 'q' 'r' 's' 't' 'u' 'v' 'w' 'x' 'y' 'z']


In [6]:
text_as_int = np.array([char2idx[c] for c in text])

print(text_as_int[0:10])
print(text[0:10])

[18 47 56 57 58  1 15 47 58 47]
First Citi


In [7]:
def split_input_target(id_text):
    return id_text[0:-1], id_text[1:]

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
seq_length = 100
seq_dataset = char_dataset.batch(seq_length+1, drop_remainder=True)

for ch_id in char_dataset.take(2):
    print(ch_id, idx2char[ch_id.numpy()])
    
for seq_id in seq_dataset.take(2):
    print(seq_id)
    print(repr(' '.join(idx2char[seq_id.numpy()])))

tf.Tensor(18, shape=(), dtype=int32) F
tf.Tensor(47, shape=(), dtype=int32) i
tf.Tensor(
[18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56 43  1 61 43
  1 54 56 53 41 43 43 42  1 39 52 63  1 44 59 56 58 46 43 56  6  1 46 43
 39 56  1 51 43  1 57 54 43 39 49  8  0  0 13 50 50 10  0 31 54 43 39 49
  6  1 57 54 43 39 49  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10
  0 37 53 59  1], shape=(101,), dtype=int32)
'F i r s t   C i t i z e n : \n B e f o r e   w e   p r o c e e d   a n y   f u r t h e r ,   h e a r   m e   s p e a k . \n \n A l l : \n S p e a k ,   s p e a k . \n \n F i r s t   C i t i z e n : \n Y o u  '
tf.Tensor(
[39 56 43  1 39 50 50  1 56 43 57 53 50 60 43 42  1 56 39 58 46 43 56  1
 58 53  1 42 47 43  1 58 46 39 52  1 58 53  1 44 39 51 47 57 46 12  0  0
 13 50 50 10  0 30 43 57 53 50 60 43 42  8  1 56 43 57 53 50 60 43 42  8
  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 18 47 56 57 58  6  1
 63 53 59  1 49], shape=(101,), dtype=int32)
'a r e   a l l 

In [8]:
seq_dataset = seq_dataset.map(split_input_target)

for item_input, item_output in seq_dataset.take(2):
    print(item_input.numpy())
    print(item_output.numpy())

[18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56 43  1 61 43
  1 54 56 53 41 43 43 42  1 39 52 63  1 44 59 56 58 46 43 56  6  1 46 43
 39 56  1 51 43  1 57 54 43 39 49  8  0  0 13 50 50 10  0 31 54 43 39 49
  6  1 57 54 43 39 49  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10
  0 37 53 59]
[47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56 43  1 61 43  1
 54 56 53 41 43 43 42  1 39 52 63  1 44 59 56 58 46 43 56  6  1 46 43 39
 56  1 51 43  1 57 54 43 39 49  8  0  0 13 50 50 10  0 31 54 43 39 49  6
  1 57 54 43 39 49  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10  0
 37 53 59  1]
[39 56 43  1 39 50 50  1 56 43 57 53 50 60 43 42  1 56 39 58 46 43 56  1
 58 53  1 42 47 43  1 58 46 39 52  1 58 53  1 44 39 51 47 57 46 12  0  0
 13 50 50 10  0 30 43 57 53 50 60 43 42  8  1 56 43 57 53 50 60 43 42  8
  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 18 47 56 57 58  6  1
 63 53 59  1]
[56 43  1 39 50 50  1 56 43 57 53 50 60 43 42  1 56 39 58 46 43 56  1 58
 53  1 42

In [9]:
batch_size = 64
buffer_size = 10000

seq_dataset = seq_dataset.shuffle(buffer_size).batch(batch_size, drop_remainder=True)

In [10]:
vocab_size = len(vocab)
embedding_dim = 256
run_units = 1024

def build_model(vocab_size, embedding_dim, run_units, batch_size):
    model = keras.models.Sequential([
        keras.layers.Embedding(
            vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),
        keras.layers.LSTM(
            units=run_units, return_sequences=True, 
            stateful=True, recurrent_initializer='glorot_uniform'),
        keras.layers.Dense(vocab_size)
    ])
    return model

model = build_model(vocab_size, embedding_dim, run_units, batch_size)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          5246976   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [11]:
for input_example_batch, target_example_batch in seq_dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape)

(64, 100, 65)


In [12]:
sample_indices = tf.random.categorical(
    logits=example_batch_predictions[0], num_samples=1)
# print(sample_indices)

sample_indices = tf.squeeze(sample_indices, axis=-1)
print(sample_indices)

tf.Tensor(
[12  4  4 59 55 43 42 37 61 25 30 24 49 28 10 20 45 64 18 17 15 30  6 64
 49 27 25 24 64 45  2 42 56 44 27 45 12 41 18  6 50 64 54 43  5 53 43  9
  2 35  4 61 61 20 22 61  4  6 27 11 33 24 47 62  0 37 41 55 45  2  8 21
  8 42 50 22 43 63 52 26 21 26 41 36 34 16 58 46 34 41 27 32 62 35 44 50
 58 34  4  0], shape=(100,), dtype=int64)


In [13]:
print("input: ", repr("".join(idx2char[input_example_batch[0]])))
print()
print("output: ", repr("".join(idx2char[target_example_batch[0]])))
print()
print("predictions: ", repr("".join(idx2char[sample_indices])))

input:  "stinctly billeted, already in the entertainment,\nand to be on foot at an hour's warning.\n\nRoman:\nI a"

output:  "tinctly billeted, already in the entertainment,\nand to be on foot at an hour's warning.\n\nRoman:\nI am"

predictions:  "?&&uqedYwMRLkP:HgzFECR,zkOMLzg!drfOg?cF,lzpe'oe3!W&wwHJw&,O;ULix\nYcqg!.I.dlJeynNINcXVDthVcOTxWfltV&\n"


In [14]:
def loss(labels, logits):
    return keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

model.compile(optimizer='adam', loss=loss)

example_loss = loss(target_example_batch, example_batch_predictions)
print(example_loss.shape)
print(example_loss.numpy().mean())

(64, 100)
4.1742573


In [15]:
%%time
output_dir = "./text_generation_checkpoints"
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
    
checkpoint_prefix = os.path.join(output_dir, 'ckpt_{epoch}')
checkpoint_callbacks = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix, save_weights_only=True)

epochs = 10
history = model.fit(seq_dataset, epochs=epochs, callbacks=[checkpoint_callbacks])

Epoch 1/10
172/172 [==============================] - 290s 2s/step - loss: 2.6720
Epoch 2/10
172/172 [==============================] - 348s 2s/step - loss: 2.0187
Epoch 3/10
172/172 [==============================] - 345s 2s/step - loss: 1.7665
Epoch 4/10
172/172 [==============================] - 338s 2s/step - loss: 1.6067
Epoch 5/10
172/172 [==============================] - 344s 2s/step - loss: 1.5028
Epoch 6/10
172/172 [==============================] - 346s 2s/step - loss: 1.4322
Epoch 7/10
172/172 [==============================] - 346s 2s/step - loss: 1.3794
Epoch 8/10
172/172 [==============================] - 323s 2s/step - loss: 1.3382
Epoch 9/10
172/172 [==============================] - 280s 2s/step - loss: 1.3007
Epoch 10/10
172/172 [==============================] - 244s 1s/step - loss: 1.2664
Wall time: 53min 49s


In [16]:
tf.train.latest_checkpoint(output_dir)

'./text_generation_checkpoints\\ckpt_10'

In [17]:
model2 = build_model(vocab_size, embedding_dim, run_units, batch_size=1)
model2.load_weights(tf.train.latest_checkpoint(output_dir))
model2.build(tf.TensorShape([1, None]))
model2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            16640     
_________________________________________________________________
lstm_1 (LSTM)                (1, None, 1024)           5246976   
_________________________________________________________________
dense_1 (Dense)              (1, None, 65)             66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [21]:
def generate_text(model, start_string, num_generate=1000):
    input_eval = [char2idx[ch] for ch in start_string]
    input_eval = tf.expand_dims(input_eval, 0)
    
    text_generated = []
    model.reset_states()
    temperature = 0.5
    
    for _ in range(num_generate):
        predictions = model(input_eval)
        predictions = predictions / temperature
        predictions = tf.squeeze(predictions, 0)
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()
        text_generated.append(idx2char[predicted_id])
        input_eval = tf.expand_dims([predicted_id], 0)        
    return start_string + ''.join(text_generated)

new_text = generate_text(model2, "all: ")
print(new_text)

all: the deed and sweet bitter,
That it us all the law will have his hand,
And that the more he did remember what I say,
That have I then must be perform'd.

KING RICHARD III:
But when he sits speak as she did speak with me;
The sight of the street hath heard him him,
And he that warrant him to the people,
So that your absence of his shame,
That my are all as his dear love and watch,
That she shall have the seas that made the king.

KING RICHARD III:
Let me as for thee that with the business.

KING RICHARD III:
All have I have a man and the better than a court,
And say 'tis banish'd and he shall shame thee speak.

LADY CAPULET:
That I was banish'd now, the sea with the state.

KING RICHARD III:
My lord, you shall be for more again your hands.

KING RICHARD II:
Then shall I see that will achore him to the common people.

ANGELO:
How now, what news?

GRUMIO:
A better end, you shall be forgot;
And that we shall have be the best way.

JULIET:
O thou hast said the heaves of the country: but